In [5]:
import os
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
from pytorch_lightning.callbacks import EarlyStopping




# Добавим трекинг

In [1]:
!pip install clearml

In [2]:
from clearml import Task
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=E2SEALD7E632F25WHX9Y4LXUO9V13K
%env CLEARML_API_SECRET_KEY=b8tZG6KHZ-WR_Ssei-ymbOAQGQ82td8EUm37SjOXAEPRXpJBXn9MkCaMj4ZMvxyBYcU

task = Task.init(project_name="", task_name="CT_class")

env: CLEARML_WEB_HOST=https://app.clear.ml/
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=E2SEALD7E632F25WHX9Y4LXUO9V13K
env: CLEARML_API_SECRET_KEY=b8tZG6KHZ-WR_Ssei-ymbOAQGQ82td8EUm37SjOXAEPRXpJBXn9MkCaMj4ZMvxyBYcU
2024-10-03 11:32:03,458 - clearml.Repository Detection - WARNING - Jupyter Notebook auto-logging failed, could not access: /kaggle/working/__notebook_source__.ipynb


Jupyter Notebook auto-logging failed, could not access: /kaggle/working/__notebook_source__.ipynb


ClearML Task: created new task id=d1d24a82d5a643368306618f9b0b4d4b
ClearML results page: https://app.clear.ml/projects/a640a1a4c9ae4ba8be2bcb1621c14098/experiments/d1d24a82d5a643368306618f9b0b4d4b/output/log


/opt/conda/lib/python3.10/site-packages/clearml/utilities/process/mp.py:629: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



In [6]:

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(30),
        transforms.ToTensor()

    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
  
}


In [7]:
class_mapping = {
    'FRONTAL': 0,
    'LATERAL': 1,
    'TRASH': 2
      }

# Определяем датасет

In [8]:
class XRayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
      
       
        

        for label in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, label)
            if os.path.isdir(class_dir):
                for img_file in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_file)
                    if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                        self.image_paths.append(img_path)
                        self.labels.append(label)
   
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:        
            image = Image.open(img_path).convert("RGB")    
        except Exception as e:        
            print(f"Ошибка при загрузке {img_path}: {e}")
            return None, None  # Возвращаем None в случае ошибки
        if self.transform:
            image = self.transform(image)

        label = self.labels[idx]
        
        if label not in class_mapping:
            print(f"Unknown label: {label}")
        label_index = class_mapping[label] 
        return image, label_index
    def get_image_id(self, img_path):
        """Получает имя файла (идентификатор изображения) из полного пути."""
        return os.path.basename(img_path)

# Определяем DataModule
class XRayDataModule(pl.LightningDataModule):
    def __init__(self, train_dir, val_dir, batch_size=32, num_workers=4):
        super(XRayDataModule, self).__init__()
        self.train_dir = train_dir
        self.val_dir = val_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_dataset = None
        self.val_dataset = None
    

    def setup(self, stage=None):
        self.train_dataset = XRayDataset(self.train_dir, transform=data_transforms['train'])
        self.val_dataset = XRayDataset(self.val_dir, transform=data_transforms['val'])

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=self.num_workers)


# Определяем класс классификатора

In [9]:

class XRayClassifier(pl.LightningModule):
    def __init__(self, num_classes=3, learning_rate=0.001):
        super(XRayClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()
        self.learning_rate = learning_rate 

        

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.loss_fn(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.loss_fn(outputs, labels)
        self.log('val_loss', loss)
        preds = torch.argmax(outputs, dim=1)
        acc = (preds == labels).float().mean()
        self.log('val_accuracy', acc)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

#  Grid Search 

In [10]:
from sklearn.model_selection import ParameterGrid
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl

param_grid = {
    'learning_rate': [0.0001,0.00001],
    'batch_size': [16,32],
    'num_epochs': [20]
}

grid = ParameterGrid(param_grid)

best_model = None
best_val_loss = float('inf')
results = []  

for params in grid:
    print(f"Training with params: {params}")

    data_module = XRayDataModule(
        train_dir='/kaggle/input/t-projection/train/train',
        val_dir='/kaggle/input/t-projection/validation/validation',
        batch_size=params['batch_size'],
        num_workers=4,
        
    )
    data_module.setup()

    xray_classifier = XRayClassifier(num_classes=3, learning_rate=params['learning_rate'])
    
    # Коллбэк для ранней остановки
    early_stopping = EarlyStopping(
        monitor='val_loss',   # Мониторим валидационный loss
        patience=3,           # Останавливаем обучение, если валидационный loss не улучшается в течение 3 эпох
        mode='min',           # Мы хотим минимизировать валидационный loss
        verbose=True
       )

    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        filename='sample-{epoch:02d}-{val_loss:.2f}',
        save_top_k=3,
        mode='min',
        dirpath='/kaggle/working/checkpoints'
    )

    tensorboard_logger = TensorBoardLogger("/kaggle/working/logs", name="my_model")

    trainer = pl.Trainer(
        max_epochs=params['num_epochs'],
        callbacks=[checkpoint_callback, early_stopping ],
        enable_progress_bar=True,
        logger=tensorboard_logger
    )

    trainer.fit(xray_classifier, data_module)

    val_loss = trainer.callback_metrics['val_loss'].item()
    val_accuracy = trainer.callback_metrics['val_accuracy'].item()

    results.append({
        'learning_rate': params['learning_rate'],
        'batch_size': params['batch_size'],
        'num_epochs': params['num_epochs'],
        'val_loss': val_loss,
        'val_accuracy': val_accuracy,
    })

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = xray_classifier

print(f"Best validation loss: {best_val_loss}")

print("\nResults for all parameter combinations:")
for result in results:
    print(result)


Training with params: {'batch_size': 16, 'learning_rate': 0.0001, 'num_epochs': 20}


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /kaggle/working/checkpoints exists and is not empty.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Best validation loss: 0.18988138437271118

Results for all parameter combinations:
{'learning_rate': 0.0001, 'batch_size': 16, 'num_epochs': 20, 'val_loss': 0.27968981862068176, 'val_accuracy': 0.9039039015769958}
{'learning_rate': 1e-05, 'batch_size': 16, 'num_epochs': 20, 'val_loss': 0.21972712874412537, 'val_accuracy': 0.9029029011726379}
{'learning_rate': 0.0001, 'batch_size': 32, 'num_epochs': 20, 'val_loss': 0.3064603805541992, 'val_accuracy': 0.8958958983421326}
{'learning_rate': 1e-05, 'batch_size': 32, 'num_epochs': 20, 'val_loss': 0.18988138437271118, 'val_accuracy': 0.9119119048118591}


Лучшее значение loss val: 0.062
Комбинация гиперпараметров:
learning_rate: 0.0001
batch_size: 32
num_epochs: 20
Точность: 0.9819

In [11]:
best_model

XRayClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

# Сохраним веса

In [12]:
best_model_weights_path = '/kaggle/working/best_model_weights.pth'
torch.save(best_model.state_dict(), best_model_weights_path)
print(f"Weights saved to {best_model_weights_path}")


Weights saved to /kaggle/working/best_model_weights.pth


In [13]:
best_model_path = '/kaggle/working/best_model.pth'
torch.save(best_model, best_model_path)
print(f"Model saved to {best_model_path}")


Model saved to /kaggle/working/best_model.pth


In [ ]:
def visualize_data(data_module, num_images=16):
    train_loader = data_module.train_dataloader()
    images, labels = next(iter(train_loader))
    class_mapping = {0: 'FRONTAL', 1: 'LATERAL', 2: 'TRASH'}
    labels = [class_mapping[label.item()] for label in labels]
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # Преобразуем тензор (C, H, W) в (H, W, C)
        plt.title(labels[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_data(data_module)


In [ ]:
from collections import Counter

def check_class_distribution(data_module):
    train_loader = data_module.train_dataloader()
    labels = []
    for _, batch_labels in train_loader:
        labels.extend(batch_labels.cpu().numpy())

    class_count = Counter(labels)
    print("Распределение классов в тренировочном наборе:")
    for class_label, count in class_count.items():
        print(f'Класс {class_label}: {count} изображений')
check_class_distribution(data_module)


# Validation результаты

In [16]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

# Загрузка модели
model = XRayClassifier(num_classes=3)  # Создаем экземпляр модели
best_model_path = '/kaggle/input/test15/pytorch/default/1/best_model_weights.pth'

# Загрузка сохраненных весов
model.load_state_dict(torch.load(best_model_path))

# Перемещаем модель на устройство (GPU или CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Перевод модели в режим вывода (inference)
model.eval()

# Валидационный лоадер
validation_loader = data_module.val_dataloader()

def inference_and_save_csv(model, dataloader, output_file):
    model.eval()  # Перевод модели в режим вывода
    results = []
    
    dataset = dataloader.dataset

    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(tqdm(dataloader)):  # Обертка с tqdm для отслеживания прогресса
            images = images.to(device)

            outputs = model(images)
            activations = torch.softmax(outputs, dim=1).cpu().numpy()
            
            for i in range(len(images)):
                image_id = dataset.get_image_id(dataset.image_paths[batch_idx * dataloader.batch_size + i])
                
                results.append({
                    "image_id": image_id,
                    "frontal": activations[i][0],
                    "lateral": activations[i][1],
                    "trash": activations[i][2]
                })
    
    print(f"Всего результатов для сохранения: {len(results)}")
    
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False, header=True)
    print(f"Результаты сохранены в {output_file}")

output_csv_path = "/kaggle/working/classification_results.csv"
inference_and_save_csv(model, validation_loader, output_csv_path)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning:

The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning:

Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.

/opt/conda/lib/python3.10/site-packages/clearml/binding/frameworks/pytorch_bind.py:277: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-mo

  0%|          | 0/32 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



Всего результатов для сохранения: 999
Результаты сохранены в /kaggle/working/classification_results.csv


Выявим ошибки

In [17]:

frontal_dir = '/kaggle/input/t-projection/validation/validation/FRONTAL'
lateral_dir = '/kaggle/input/t-projection/validation/validation/LATERAL'
trash_dir = '/kaggle/input/t-projection/validation/validation/TRASH'


true_labels = {}


for label, directory in zip(['frontal', 'lateral', 'trash'], [frontal_dir, lateral_dir, trash_dir]):
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            true_labels[filename] = label


predictions_df = pd.read_csv(output_csv_path)


incorrect_count = 0

for _, row in predictions_df.iterrows():
    image_id = row['image_id']
    predicted_label = 'frontal' if row['frontal'] > max(row['lateral'], row['trash']) else ('lateral' if row['lateral'] > row['trash'] else 'trash')
    
    true_label = true_labels.get(image_id, None)
    
    if true_label and predicted_label != true_label:
        incorrect_count += 1

print(f"Количество неверных предсказаний: {incorrect_count}")


Количество неверных предсказаний: 15


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# Пути к директориям с изображениями
frontal_dir = '/kaggle/input/t-projection/validation/validation/FRONTAL'
lateral_dir = '/kaggle/input/t-projection/validation/validation/LATERAL'
trash_dir = '/kaggle/input/t-projection/validation/validation/TRASH'

true_labels = {}

for label, directory in zip(['frontal', 'lateral', 'trash'], [frontal_dir, lateral_dir, trash_dir]):
    for filename in os.listdir(directory):
        if filename.endswith('.png'):
            true_labels[filename] = label

predictions_df = pd.read_csv(output_csv_path)

incorrect_predictions = []

for _, row in predictions_df.iterrows():
    image_id = row['image_id']
    predicted_label = 'frontal' if row['frontal'] > max(row['lateral'], row['trash']) else ('lateral' if row['lateral'] > row['trash'] else 'trash')
    
    true_label = true_labels.get(image_id, None)
    
    if true_label and predicted_label != true_label:
        incorrect_predictions.append((image_id, true_label, predicted_label))

num_images = min(len(incorrect_predictions), 10)  # Ограничение до 10 изображений
plt.figure(figsize=(15, 7 * (num_images // 2 + 1)))

for i, (image_id, true_label, predicted_label) in enumerate(incorrect_predictions[:num_images]):
    image_path = os.path.join(frontal_dir if true_label == 'frontal' else lateral_dir if true_label == 'lateral' else trash_dir, image_id)
    image = Image.open(image_path)

    plt.subplot(num_images // 3 + 1, 3, i + 1)
    plt.imshow(image)
    plt.title(f"True: {true_label}, Pred: {predicted_label}\nFilename: {image_id}")
    plt.axis('off')

plt.tight_layout()
plt.show()
